### **Librerías**

In [ ]:
import json
import numpy as np
import os
import pandas as pd
import pprint as pp
import re
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from constants import DATA_DIR

from IPython.display import clear_output
from statsmodels.tsa.arima_process import arma_generate_sample

np.random.seed(13)

### **Columnas para los DataFrames**

In [ ]:
COLUMNS = ['sim', 'id', 'inicio_prog', 't', 'y', 'y_cf', 'tratado', 'control']

### **Directorio base de los datos**

In [ ]:
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
else:
    print(f"The directory {DATA_DIR} already exists")

### **Número de grupo**

In [ ]:
# Get the number of the last group generated
groups_dirs = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
pattern = r'Grupo(\d+)'

numbers = []
for dir in groups_dirs:
    match = re.search(pattern, dir)
    if match:
        numbers.append(int(match.group(1)))

if numbers:
    group_number = max(numbers) + 1
    print(f"The new group number is: {group_number}")
else:
    print("No folders with the pattern 'GrupoN' found. Setting it to 1...")
    group_number = 1

GROUP_PATH = os.path.join(DATA_DIR, f"Grupo{group_number}")
print(f"Creating the group directory: {GROUP_PATH}")
os.makedirs(GROUP_PATH)

### **Parámetros**
Para más información sobre qué representa cada parámetro, leer el archivo `README.md` de este directorio.

#### **Parámetros fijos**

In [ ]:
ini = 200

phiNini = 0.90
phiT = 0.90
phiC = 0.90
phiTra = 0.90
MeanFEN = 10.0
MeanFET = 10.0
MeanFEC = 10.0

NivelN = MeanFEN
NivelT = MeanFET
NivelC = MeanFEC
ImpactoProporcional = 0.05
ImpactoNominal = NivelC * ImpactoProporcional
STDImpacto = 0.05

dependence = True

porcentaje  = 0.05
hetecohorte = 1

StdErrorSerie = 5

#### **Cargamos parámetros variables**

In [ ]:
PARAMS_PATH = os.path.join(os.getcwd(), 'params.json')
with open(PARAMS_PATH, 'r') as f:
    params = json.load(f)

n_simulations = params['n_simulations']

n_total = params['n_total']
treated_pctg = params['treated_pctg']
control_pctg = params['control_pctg']

n_treated = round(treated_pctg * n_total)
n_control = round(control_pctg * n_total)
n_nini = n_total - (n_treated + n_control)

T = params['T']
total_periods = T + ini
first_tr_period = params['first_tr_period']

n_cohorts = params['n_cohorts']
n_per_dep = params['n_per_dep']

if n_per_dep > first_tr_period:
    raise Exception(
        "The number of depending periods is greater than the first treatment period."
        "Please correct it."
    )

pp.pp(params)

Escribimos los parámetros usados a la carpeta del nuevo grupo.

In [ ]:
params["group_number"] = group_number
group_params_path = os.path.join(GROUP_PATH, f"params_Grupo{group_number}.json")
with open(group_params_path, "w") as f:
    json.dump(params, f, indent=4)

In [ ]:
def gen_time_series_upw_trend(
    steps, n_per_dep, fixed_effect, temp_effect, treatment_start
):
    y = np.zeros(steps)
    y[0] = (
        MeanFET + fixed_effect + temp_effect + np.random.normal(0, 1) * StdErrorSerie
    )

    trend_start = treatment_start - n_per_dep
    for t in range(1, steps):
        next_value = (
            (1 - phiT) * (MeanFET + fixed_effect + temp_effect) + phiT * y[t - 1]
            + np.random.normal(0, 1) * StdErrorSerie
        )
        if t > trend_start and t < treatment_start:
            # Generate next value so that it is greater than the previous one
            while next_value <= y[t-1]:
                next_value = (
                    (1 - phiT) * (MeanFET + fixed_effect + temp_effect) + phiT * y[t-1]
                    + np.random.normal(0, 1) * StdErrorSerie
                )
        y[t] = next_value

    return y

In [ ]:
for sim in range(n_simulations):
    print(f"Simulación {sim+1} de {n_simulations}")

    YPanelTreated = np.zeros(shape=(n_treated*T, len(COLUMNS)))
    YPanelControl = np.zeros(shape=(n_control*T, len(COLUMNS)))
    YPanelNiNi = np.zeros(shape=(n_nini*T, len(COLUMNS)))

    # A time effect, one for each time step
    EfectoTemporalT = np.random.normal(0,1,T+ini)
    EfectoTemporalTMedio = EfectoTemporalT.mean()

    y_treated = np.zeros(shape=(n_treated, T))
    y_control = np.zeros(shape=(n_control, T))
    y_nini = np.zeros(shape=(n_nini, T))
    tr_starts_indexes = []

    # Amount of treated units per cohort
    n_treated_in_cohort = int(n_treated / n_cohorts) + 1

    for dataset, label in [(y_treated, 'Tratados'), (y_control, 'Controles')]:
        print(f"\tGenerando datos para {label}...")
        # A fixed effect for each unit
        EfectoFijoT = np.random.normal(0, 1, len(dataset)) + MeanFET
        for cohort in range(n_cohorts):
            # Subtract 1 to turn it 0-indexed
            tr_start_index = (ini + first_tr_period + cohort) - 1
            print(
                f"\t\tCohorte {cohort+1} de {n_cohorts}: "
                f"inicio de programa en {tr_start_index+1-ini}."
            )
            for i in range(n_treated_in_cohort):
                y_i = gen_time_series_upw_trend(
                    total_periods, n_per_dep, EfectoFijoT[i], EfectoTemporalTMedio,
                    tr_start_index
                )
                tr_starts_indexes.append(tr_start_index-ini)
                index = (cohort*n_treated_in_cohort) + i
                if index < len(dataset):
                    dataset[index, :] = y_i[ini:]
        print()

    y_counterfac = y_treated.copy()

    # Apply treatment only to the treated units
    for i in range(n_treated):
        tr_start_index = tr_starts_indexes[i]
        tr_length = T - tr_start_index
        if hetecohorte == 1:
            arparams = np.array([phiTra, 0])
            maparams = np.array([0, 0])
            arparams = np.r_[1, -arparams]
            maparams = np.r_[1, maparams]
            impact = arma_generate_sample(
                arparams, maparams, tr_length, burnin=5000
            ) + ImpactoNominal
            y_treated[i, tr_start_index:] += impact
        else:
            TraCondicion = np.array(tr_starts_indexes)
            y_treated[i, tr_start_index:] += np.random.normal(
                ImpactoNominal, STDImpacto, tr_length
            )
            TraCondicion = np.array(tr_starts_indexes)

    # Reshape the data from horizontal to vertical format with extra columns
    for panel, dataset, label in [
        (YPanelTreated, y_treated, 'Tratados'),
        (YPanelControl, y_control, 'Controles')
    ]:
        print(f"\tCambiando formato de datos para {label}...")
        n = len(dataset)
        panel[:, 0] = sim + 1
        panel[:, 6] = int(label == 'Tratados')
        panel[:, 7] = int(label == 'Controles')

        ids = np.zeros((n*T, 1))
        tr_starts = np.zeros((n*T, 1))
        steps = np.zeros((n*T, 1))
        y = np.zeros((n*T, 1))
        y_cf = np.zeros((n*T, 1))
        for i in range(n):
            ids[(i*T):((i+1)*T)] = i if label == 'Tratados' else i + n_treated
            tr_starts[(i*T):((i+1)*T)] = tr_starts_indexes[i]
            steps[(i*T):((i+1)*T)] = np.arange(T).reshape(T, 1)
            y[(i*T):((i+1)*T)] = np.reshape(dataset[i,:], (T, 1))
            y_cf[(i*T):((i+1)*T)] = (
                np.reshape(y_counterfac[i,:], (T, 1)) if label == 'Tratados' else 0
            )

        panel[:, 1] = np.reshape(ids, (n*T,))
        panel[:, 2] = np.reshape(tr_starts, (n*T,))
        panel[:, 3] = np.reshape(steps, (n*T,))
        panel[:, 4] = np.reshape(y, (n*T,))
        panel[:, 5] = np.reshape(y_cf, (n*T,))

    # Generate the data for the nini group
    EfectoFijoT = np.random.normal(0, 1, n_nini) + MeanFEN

    print(f"\tGenerando datos para NiNi...")
    for i in range(n_nini):
        y_i = np.zeros(total_periods)
        y_i[0] = (
            MeanFEN + EfectoFijoT[i] + EfectoTemporalTMedio
            + np.random.normal(0,1) * StdErrorSerie
        )
        for t in range(total_periods):
            y_i[t] = (
                (1-phiNini) * (MeanFEN + EfectoFijoT[i] + EfectoTemporalTMedio)
                + phiNini * y_i[t - 1] + np.random.normal(0, 1) * StdErrorSerie
            )
        y_nini[i,:] = y_i[ini:]

    # Reshape the data from horizontal to vertical format with extra columns
    print(f"\tCambiando formato de datos para NiNi...")
    n = n_nini
    YPanelNiNi[:, 0] = sim + 1

    ids = np.zeros((n*T, 1))
    steps = np.zeros((n*T, 1))
    y = np.zeros((n*T, 1))
    for i in range(n):
        ids[(i*T):((i+1)*T)] = i + (n_treated + n_control)
        steps[(i*T):((i+1)*T)] = np.arange(T).reshape(T, 1)
        y[(i*T):((i+1)*T)] = np.reshape(y_nini[i,:], (T, 1))

    YPanelNiNi[:, 1] = np.reshape(ids, (n*T,))
    YPanelNiNi[:, 3] = np.reshape(steps, (n*T,))
    YPanelNiNi[:, 4] = np.reshape(y, (n*T,))
    # Columns 2, 5, 6 and 7 are already filled with zeros

    print("\tGenerando DataFrames")
    df_treated = pd.DataFrame(YPanelTreated, columns=COLUMNS)
    df_control = pd.DataFrame(YPanelControl, columns=COLUMNS)
    df_nini = pd.DataFrame(YPanelNiNi, columns=COLUMNS)
    
    df = pd.concat([df_treated, df_control, df_nini])
    df['sim'] = df['sim'].astype(int)
    df['id'] = df['id'].astype(int)
    df['inicio_prog'] = df['inicio_prog'].astype(int)
    df['t'] = df['t'].astype(int)
    df['tratado'] = df['tratado'].astype(int)
    df['control'] = df['control'].astype(int)

    sim_path = os.path.join(GROUP_PATH, f"Simulacion{sim+1}.dta")
    df.to_stata(sim_path, write_index=False)
    clear_output(wait=True)

Como chequeo de sanidad sobre las simulaciones generadas, verificamos varias cosas:
* Que cada simulación tenga la cantidad total deseada de individuos (`n_total`)
* Que cada simulación tenga la cantidad total deseada de tratados (`n_treated`)
* Que cada simulación tenga la cantidad total deseada de controles (`n_control`)
* Que cada simulación tenga la cantidad total deseada de nini (`n_nini`)
* Que en los períodos relevantes (que van desde uno antes del inicio del programa
hasta el inicio del programa menos la cantiadad de períodos de dependencia):
  * Todos los valores de `y` hayan sido generados
  * Todos los valores de `y` sean distintos de cero
  * Tratados y controles tengan la tendencia creciente deseada
  * Los nini no tengan la tendencia creciente

In [ ]:
for f in os.listdir(GROUP_PATH):
    if f.endswith('.dta'):
        path = os.path.join(GROUP_PATH, f)
        df = pd.read_stata(path)
        assert(len(df['id'].unique()) == n_total)
        df_treated = df[df['tratado'] == 1]
        assert(len(df_treated['id'].unique()) == n_treated)
        df_control = df[df['control'] == 1]
        assert(len(df_control['id'].unique()) == n_control)
        df_nini = df[(df['tratado'] == 0) & (df['control'] == 0)]
        assert(len(df_nini['id'].unique()) == n_nini)
        for group_df in [df_treated, df_control, df_nini]:
            relevant_periods_df = group_df[
                (group_df['inicio_prog'] - n_per_dep <= group_df['t']) &
                (group_df['t'] <= group_df['inicio_prog'] - 1)
            ]
            for id, data in relevant_periods_df.groupby('id'):
                y = data['y'].to_numpy()
                assert(len(y) == n_per_dep)
                assert(np.all(y != 0))
                try:
                    if (group_df is df_treated) or (group_df is df_control):
                        assert(np.all(np.diff(y) > 0))
                    else:
                        assert(not(np.all(np.diff(y) > 0)))
                except AssertionError:
                    print(
                        f"Error: La serie temporal de la unidad {id} "
                        f"en el archivo {f} no tiene tendencia creciente "
                        f"en los períodos relevantes."
                    )
                    break